In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import os, sys

from statsmodels.tsa.stattools import adfuller, kpss

sys.path.append(os.path.abspath(os.path.join('../src/')))
from networks import *
from generative_models import *

import tensorflow as tf
from tensorflow.keras.utils import to_categorical

In [ ]:
%load_ext autoreload
%autoreload 2
np.set_printoptions(suppress=True)

In [ ]:
# initialize network
network = HeteroscedasticNetwork(6, 6)
network.load_weights('../model_comparison/checkpoints/varying_hyperparams_3200')

In [ ]:
# read data
data = pd.read_csv('../data/data_lexical_decision.csv', sep=',', header=0)
data.loc[data.acc == 0, 'rt'] = -data.loc[data.acc == 0, 'rt']
data.loc[:, 'stim_type'] = data.loc[:, 'stim_type'] - 1
emp_data = data[['rt', 'stim_type']].to_numpy().reshape(11, 3200, 2)
emp_data.shape

In [ ]:
# amortized inference
net_in = tf.concat((emp_data[:, :, :1], to_categorical(emp_data[:, :, 1:])), axis=-1)
post_d, post_s = network(net_in)
post_d.shape


In [ ]:
np.array(post_d.mean()[0])

In [213]:
def kpss_test(timeseries):
    print("Results of KPSS Test:")
    kpsstest = kpss(timeseries, regression="c", nlags="auto")
    kpss_output = pd.Series(
        kpsstest[0:3], index=["Test Statistic", "p-value", "Lags Used"]
    )
    for key, value in kpsstest[3].items():
        kpss_output["Critical Value (%s)" % key] = value
    print(kpss_output)

def adf_test(timeseries):
    print("Results of Dickey-Fuller Test:")
    dftest = adfuller(timeseries, autolag="AIC")
    dfoutput = pd.Series(
        dftest[0:4],
        index=[
            "Test Statistic",
            "p-value",
            "#Lags Used",
            "Number of Observations Used",
        ],
    )
    for key, value in dftest[4].items():
        dfoutput["Critical Value (%s)" % key] = value
    print(dfoutput)

In [212]:
rt = np.abs(emp_data[0, :, 0])
kpss_test(rt)

Results of KPSS Test:
Test Statistic            6.23086
p-value                   0.01000
Lags Used                25.00000
Critical Value (10%)      0.34700
Critical Value (5%)       0.46300
Critical Value (2.5%)     0.57400
Critical Value (1%)       0.73900
dtype: float64


/opt/anaconda3/envs/cognitiveModeling/lib/python3.9/site-packages/statsmodels/tsa/stattools.py:1906: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  warnings.warn(


In [214]:
adf_test(rt)

Results of Dickey-Fuller Test:
Test Statistic                -7.493450e+00
p-value                        4.442852e-11
#Lags Used                     2.100000e+01
Number of Observations Used    3.178000e+03
Critical Value (1%)           -3.432409e+00
Critical Value (5%)           -2.862450e+00
Critical Value (10%)          -2.567254e+00
dtype: float64


In [ ]:
# tests for empiric rt time series
n_sub = post_d.shape[0]
adf_data = list()
kpss_data = list()
adf_data_p_value = np.empty(n_sub)
kpss_data_p_value = np.empty(n_sub)
for sub in range(n_sub):
    # subset rt time series
    rt = np.abs(emp_data[sub, :, 0])

    # compute tests
    adf_test = adfuller(rt)
    kpss_test = kpss(rt, nlags="auto")

    # store results
    adf_data.append(adf_test)
    kpss_data.append(kpss_test)

    # store p-values separately
    adf_data_p_value[sub] = adf_test[1]
    kpss_data_p_value[sub] = kpss_test[1]

In [209]:
adf_data_p_value

array([4.44285197e-11, 2.97087065e-14, 3.63060117e-18, 1.61012454e-07,
       1.17180126e-19, 1.50037787e-15, 9.32761707e-09, 1.59047171e-07,
       3.63820101e-20, 7.45553555e-07, 4.25602098e-29])

In [210]:
kpss_data_p_value

array([0.01      , 0.01      , 0.01      , 0.01      , 0.04811223,
       0.01      , 0.01      , 0.01      , 0.01      , 0.01      ,
       0.01      ])

In [ ]:
# tests for dynamic parameters
n_sub = post_d.shape[0]
n_trim = 200
adf_params_p_value = np.empty((n_sub, 6))
kpss_params_p_value = np.empty((n_sub, 6))
for sub in range(n_sub):
    for i in range(6):
        # compute tests
        adf_test = adfuller(np.array(post_d.mean()[sub])[n_trim:, i])
        kpss_test = kpss(np.array(post_d.mean()[sub])[n_trim:, i], nlags="auto")

        # store p-values separately
        adf_params_p_value[sub, i] = adf_test[1]
        kpss_params_p_value[sub, i] = kpss_test[1]

In [ ]:
which = 2

In [ ]:
adf_params_p_value[which]

In [ ]:
kpss_params_p_value[which]

In [ ]:
sns.lineplot(np.arange(n_trim, 3200), np.array(post_d.mean()[which])[n_trim:, 5])
